In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso
from sklearn.preprocessing import RobustScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adv-house-price/sample_submission.csv
/kaggle/input/adv-house-price/data_description.txt
/kaggle/input/adv-house-price/train.csv
/kaggle/input/adv-house-price/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/adv-house-price/train.csv")
test = pd.read_csv("/kaggle/input/adv-house-price/test.csv")

In [3]:
#Dealing with object value (Data Cleaning)
#Assumtion price: C>FV>RH>RM>RL (C is NaN)
#Mapping value to each object in ascending order
mapping1 = {'FV':4, 'RH':3, 'RM':2, 'RL':1, 'C (all)':5}
df['MSZoning'] = df['MSZoning'].replace(mapping1)

In [4]:
#LotFrontage have NaN value we will replace with mean value
# Replace NaN values in a specific column with the mean value of that column
df['LotFrontage'].fillna(df['LotFrontage'].mean(), inplace=True)

In [5]:
#Street data are contain ['Pave', 'Grvl'] we check mean of each street so that I'll replace with 1 and 2
#Price: Pave>Grvl
mapping2 = {'Pave':3, 'Grvl':1}
df['Street'] = df['Street'].replace(mapping2)
df['Street']

0       3
1       3
2       3
3       3
4       3
       ..
1455    3
1456    3
1457    3
1458    3
1459    3
Name: Street, Length: 1460, dtype: int64

In [6]:
#Check for LotShape data ['Reg', 'IR1', 'IR2', 'IR3']
#Price: Reg<IR1<IR2=IR3
mapping3 = {'Reg':1, 'IR1':2, 'IR2':3, 'IR3':4}
df['LotShape'] = df['LotShape'].replace(mapping3)
df['LotShape']

0       1
1       1
2       2
3       2
4       2
       ..
1455    1
1456    1
1457    1
1458    1
1459    1
Name: LotShape, Length: 1460, dtype: int64

In [7]:
#Checking for LandContour ['Lvl', 'Bnk', 'HLS', 'Low']
#Price: Bnk<Lvl<Low<HLS
mapping4 = {'Bnk':1, 'Lvl':2, 'Low':3, 'HLS':4}
df['LandContour'] = df['LandContour'].replace(mapping4)
df['LandContour']

0       2
1       2
2       2
3       2
4       2
       ..
1455    2
1456    2
1457    2
1458    2
1459    2
Name: LandContour, Length: 1460, dtype: int64

In [8]:
#Checking for Utilities column ['Allpub', 'NoSeWa']
mapping5 = {'AllPub':1, 'NoSeWa':0}
df['Utilities'] = df['Utilities'].replace(mapping5)
df['Utilities']

0       1
1       1
2       1
3       1
4       1
       ..
1455    1
1456    1
1457    1
1458    1
1459    1
Name: Utilities, Length: 1460, dtype: int64

In [9]:
Mapping_Qual = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0}
df['ExterQual'] = df['ExterQual'].replace(Mapping_Qual)
df['ExterCond'] = df['ExterCond'].replace(Mapping_Qual)
df['BsmtQual'] = df['BsmtQual'].replace(Mapping_Qual)
df['BsmtCond'] = df['BsmtCond'].replace(Mapping_Qual)
df['HeatingQC'] = df['HeatingQC'].replace(Mapping_Qual)
df['KitchenQual'] = df['KitchenQual'].replace(Mapping_Qual)
df['FireplaceQu'] = df['FireplaceQu'].replace(Mapping_Qual)
df['GarageQual'] = df['GarageQual'].replace(Mapping_Qual)
df['GarageCond'] = df['GarageCond'].replace(Mapping_Qual)
df['PoolQC'] = df['PoolQC'].replace(Mapping_Qual)
Mapping_Exposure = {'Gd':5, 'Av':4, 'Mn':3, 'No':2, 'Na':1}
df['BsmtExposure'] = df['BsmtExposure'].replace(Mapping_Exposure)
Mapping_Fintype = {'GLQ':7, 'ALQ':6, 'BLQ':5, 'Rec':4, 'LwQ':3, 'Unf':2, 'NA':1}
df['BsmtFinType1'] = df['BsmtFinType1'].replace(Mapping_Fintype)
df['BsmtFinType2'] = df['BsmtFinType2'].replace(Mapping_Fintype)
df.loc[:,'Alley'].fillna(3, inplace=True)
df['Alley'] = df['Alley'].replace({'Grvl':1, 'Pave':2})
#Price: Inside<Corner<FR2<FR3<CulDSac
Mapping_LotConfig = {'Inside':1, 'Corner':2, 'FR2':3, 'FR3':4, 'CulDSac':5}
df['LotConfig'] = df['LotConfig'].replace(Mapping_LotConfig)
#plt.bar(df['LandSlope'], df['SalePrice'])
Mapping_LandSlope = {'Gtl':3, 'Mod':2, 'Sev':1}
df['LandSlope'] = df['LandSlope'].replace(Mapping_LandSlope)
#plt.figure(figsize=(50,20))
#plt.xticks(fontsize=24)
#plt.bar(df['Condition2'], df['SalePrice'], width=0.1)
#Price: NoRidge>NridgHt>StoneBr>OldTown>Somerst=CollgCr>Veenker>Crawfor>Timber>Gilbert>Sawyer=Edwards=ClearCr>NWAmes>Mitchel>Blmngtn>BrkSide=SWISU>IDOTRR=Blueste=NPkVill>BrDale
Mapping_neightb = {'BrDale':1, 'MeadowV':2, 'NPkVill':2, 'Blueste':2, 'IDOTRR':2, 'Sawyer':2, 'SWISU':3, 'BrkSide':3, 'Blmngtn':4, 'Mitchel':5, 'NAmes':6, 'NWAmes':6, 'ClearCr':7, 'Edwards':8, 'SawyerW':8, 'Gilbert':9, 'Timber':10, 'Crawfor':11, 'Veenker':12, 'CollgCr':13, 'Somerst':13, 'OldTown':14, 'StoneBr':15, 'NridgHt':16, 'NoRidge':17}
df['Neighborhood'] = df['Neighborhood'].replace(Mapping_neightb)
#Price condition:RRAe<RRNe<Feedr<RRNn<PosA<PosN<RRAn<Artery<Norm
Mapping_condition1 = {'RRAe':1, 'RRNe':2, 'Feedr':3, 'RRNn':4, 'PosA':5, 'PosN':6, 'RRAn':7, 'Artery':8, 'Norm':9}
df['Condition1'] = df['Condition1'].replace(Mapping_condition1)
Mapping_condition2 = {'Artery':1, 'RRNn':2, 'RRAn':3, 'Feedr':4, 'RRAe':5, 'PosA':6, 'PosN':7, 'Norm':8}
df['Condition2'] = df['Condition2'].replace(Mapping_condition2)
#plt.bar(df['HouseStyle'], df['SalePrice'], width=0.1)
Mapping_Bldg = {'Duplex':1, '2fmCon':2, 'Twnhs':3, 'TwnhsE':4, '1Fam':5}
df['BldgType'] = df['BldgType'].replace(Mapping_Bldg)
Mapping_housestyle = {'1.5Unf':1, 'SFoyer':2, '2.5Unf':3, 'SLvl':4, '1.5Fin':5, '2.5Fin':6, '1Story':7, '2Story':8}
df['HouseStyle'] = df['HouseStyle'].replace(Mapping_housestyle)
#plt.bar(df['MasVnrType'], df['SalePrice'], width=0.1)
Mapping_RoofStyle = {'Gambrel':1, 'Mansard':1, 'Flat':1, 'Shed':1, 'Gable':2, 'Hip':2}
df['RoofStyle'] = df['RoofStyle'].replace(Mapping_RoofStyle)
Mapping_RoofMatl = {'Metal':1, 'Roll':1, 'ClyTile':1, 'WdShake':2, 'Membran':2, 'Tar&Grv':2, 'CompShg':3, 'WdShngl':3}
df['RoofMatl'] = df['RoofMatl'].replace(Mapping_RoofMatl)
Mapping_exterior1 = {'BrkComm':1, 'AsphShn':1, 'CBlock':1, 'AsbShng':1, 'Stone':2, 'ImStucc':2, 'Stucco':2, 'Plywood':2, 'WdShing':2, 'BrkFace':2, 'MetalSd':2, 'CemntBd':3, 'VinylSd':3, 'Wd Sdng':4, 'HdBoard':4}
df['Exterior1st'] = df['Exterior1st'].replace(Mapping_exterior1)
Mapping_exterior2 = {'Brk Cmn':1, 'AsphShn':1, 'AsbShng':1, 'Stone':1, 'CBlock':1, 'Other':2, 'Plywood':2,'Wd Shng':2 ,'Wd Sdng':3, 'BrkFace':2, 'Stucco':2, 'MetalSd':2, 'VinylSd':3, 'CmentBd':3, 'HdBoard':3, 'ImStucc':3}
df['Exterior2nd'] = df['Exterior2nd'].replace(Mapping_exterior2)
df['MasVnrType'].fillna(3, inplace=True)
Mapping_MasVnrType = {'BrkFace':3, 'Stone':2, 'BrkCmn':1}
df['MasVnrType'] = df['MasVnrType'].replace(Mapping_MasVnrType)
df['MasVnrArea'].fillna(0, inplace=True)
#plt.bar(df['Foundation'], df['SalePrice'], width=0.1)
Mapping_Foundation = {'Wood':1, 'Slab':1, 'Stone':1, 'BrkTil':2, 'CBlock':2, 'PConc':3}
df['Foundation'] = df['Foundation'].replace(Mapping_Foundation)
df['BsmtQual'].fillna(0, inplace=True)
df['BsmtCond'].fillna(0, inplace=True)
df['BsmtExposure'].fillna(0, inplace=True)
df['BsmtFinType1'].fillna(0, inplace=True)
df['BsmtFinType2'].fillna(0, inplace=True)
#plt.bar(df['Electrical'], df['SalePrice'], width=0.1)
mapping_heating = {'Grav':1, 'Wall':1, 'OthW':1, 'Floor':1, 'GasW':2, 'GasA':3}
df['Heating'] = df['Heating'].replace(mapping_heating)
df['CentralAir'] = df['CentralAir'].replace({'N':0, 'Y':1})
df['Electrical'].fillna('None', inplace=True)
#plt.bar(df['Functional'], df['SalePrice'], width=0.1)
df['Electrical'] = df['Electrical'].replace({'FuseF':1, 'FuseA':1, 'FuseP':1, 'Mix':1, 'None':1, 'SBrkr':2})
df['Functional'] = df['Functional'].replace({'Maj2':1, 'Sev':1, 'Min1':2, 'Maj1':2, 'Min2':2, 'Mod':3, 'Typ':4})
df['FireplaceQu'].fillna(0, inplace=True)
df['GarageType'].fillna('None', inplace=True)
df['GarageType'] = df['GarageType'].replace({'CarPort':1, 'None':1, '2Types':1, 'Basment':1, 'Detchd':2, 'BuiltIn':2, 'Attchd':3})
df['GarageFinish'].fillna('None', inplace=True)
#plt.bar(df['GarageFinish'], df['SalePrice'], width=0.1)
df['GarageFinish'] = df['GarageFinish'].replace({'None':0, 'RFn':1, 'Unf':1, 'Fin':1})
df['GarageYrBlt'].fillna(0, inplace=True)
df['GarageQual'].fillna(0, inplace=True)
df['GarageCond'].fillna(0, inplace=True)
#plt.bar(df['PavedDrive'], df['SalePrice'], width=0.1)
df['PavedDrive'] = df['PavedDrive'].replace({'Y':1, 'N':0, 'P':0})
df['PoolQC'].fillna(0, inplace=True)
df['Fence'].fillna('None', inplace=True)
#plt.bar(df['Fence'], df['SalePrice'], width=0.1)
df['Fence'] = df['Fence'].replace({'GdWo':1, 'GdPrv':1, 'MnWw':1, 'MnPrv':2, 'None':2})
df['MiscFeature'].fillna('None', inplace=True)
df['MiscFeature'] = df['MiscFeature'].replace({'None':1, 'Shed':0, 'Gar2':0, 'Othr':0, 'TenC':0})
#plt.bar(df['SaleCondition'], df['SalePrice'], width=0.1)
df['SaleType'] = df['SaleType'].replace({'COD':1, 'ConLD':1, 'CWD':1, 'ConLw':1, 'Con':1, 'Oth':1, 'ConLI':1, 'New':2, 'WD':2})
df['SaleCondition'] = df['SaleCondition'].replace({'AdjLand':1, 'Alloca':1, 'Family':1, 'Partial':2, 'Abnorml':2, 'Normal':2})
df['MSSubClass'] = df['MSSubClass'].replace({60:3, 20:3, 50:2, 75:2, 120:2, 30:1, 35:1, 40:1, 45:1, 70:1, 80:1, 85:1, 90:1, 160:1, 190:1, 180:1})


In [10]:
#Linear regression with 81 features 
X = df.iloc[:,1:-1]
Y = np.log1p(df.iloc[:,-1:])
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [11]:
# Create a linear regression model
robust_scaler = RobustScaler()
x_train_scaled = robust_scaler.fit_transform(x_train)
x_test_scaled = robust_scaler.fit_transform(x_test)
model_lin = LinearRegression()
model_lasso = Lasso(alpha=0.0001, random_state=42)
# Train the model on the scaled training set
model_lin.fit(x_train_scaled, y_train)
model_lasso.fit(x_train_scaled, y_train)

# Make predictions on the scaled test set
y_pred_lin = model_lin.predict(x_test_scaled)
y_pred_lasso = model_lasso.predict(x_test_scaled)

# Calculate R2 score
mse1 = mean_squared_error(y_test, y_pred_lin)
rmse1 = np.sqrt(mse1)
mse2 = mean_squared_error(y_test, y_pred_lasso)
rmse2 = np.sqrt(mse2)
r2_1 = r2_score(y_test, y_pred_lin)
r2_2 = r2_score(y_test, y_pred_lasso)

print("Root Mean Squared Error of linear regression:", rmse1)
print("Root Mean Squared Error of lasso regression:", rmse2)
print("R2 Error of linear regression:", r2_1)
print("R2 Error of lasso regression:", r2_2)

Root Mean Squared Error of linear regression: 0.14253148198571966
Root Mean Squared Error of lasso regression: 0.14320569900028804
R2 Error of linear regression: 0.880247870039672
R2 Error of lasso regression: 0.8791122629136476


In [12]:
#Submission Part
mapping5 = {'AllPub':1, 'NoSeWa':0}
test['Utilities'] = test['Utilities'].replace(mapping5)
mapping4 = {'Bnk':1, 'Lvl':2, 'Low':3, 'HLS':4}
test['LandContour'] = test['LandContour'].replace(mapping4)
mapping3 = {'Reg':1, 'IR1':2, 'IR2':3, 'IR3':4}
test['LotShape'] = test['LotShape'].replace(mapping3)
mapping2 = {'Pave':3, 'Grvl':1}
test['Street'] = test['Street'].replace(mapping2)
mapping1 = {'FV':4, 'RH':3, 'RM':2, 'RL':1, 'C (all)':5}
test['MSZoning'] = test['MSZoning'].replace(mapping1)
test['LotFrontage'].fillna(test['LotFrontage'].mean(), inplace=True)
Mapping_Qual = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0}
test['ExterQual'] = test['ExterQual'].replace(Mapping_Qual)
test['ExterCond'] = test['ExterCond'].replace(Mapping_Qual)
test['BsmtQual'] = test['BsmtQual'].replace(Mapping_Qual)
test['BsmtCond'] = test['BsmtCond'].replace(Mapping_Qual)
test['HeatingQC'] = test['HeatingQC'].replace(Mapping_Qual)
test['KitchenQual'] = test['KitchenQual'].replace(Mapping_Qual)
test['FireplaceQu'] = test['FireplaceQu'].replace(Mapping_Qual)
test['GarageQual'] = test['GarageQual'].replace(Mapping_Qual)
test['GarageCond'] = test['GarageCond'].replace(Mapping_Qual)
test['PoolQC'] = test['PoolQC'].replace(Mapping_Qual)
Mapping_Exposure = {'Gd':5, 'Av':4, 'Mn':3, 'No':2, 'Na':1}
test['BsmtExposure'] = test['BsmtExposure'].replace(Mapping_Exposure)
Mapping_Fintype = {'GLQ':7, 'ALQ':6, 'BLQ':5, 'Rec':4, 'LwQ':3, 'Unf':2, 'NA':1}
test['BsmtFinType1'] = test['BsmtFinType1'].replace(Mapping_Fintype)
test['BsmtFinType2'] = test['BsmtFinType2'].replace(Mapping_Fintype)
test.loc[:,'Alley'].fillna(3, inplace=True)
test['Alley'] = test['Alley'].replace({'Grvl':1, 'Pave':2})
#Price: Inside<Corner<FR2<FR3<CulDSac
Mapping_LotConfig = {'Inside':1, 'Corner':2, 'FR2':3, 'FR3':4, 'CulDSac':5}
test['LotConfig'] = test['LotConfig'].replace(Mapping_LotConfig)
#plt.bar(df['LandSlope'], df['SalePrice'])
Mapping_LandSlope = {'Gtl':3, 'Mod':2, 'Sev':1}
test['LandSlope'] = test['LandSlope'].replace(Mapping_LandSlope)
#plt.figure(figsize=(50,20))
#plt.xticks(fontsize=24)
#plt.bar(df['Condition2'], df['SalePrice'], width=0.1)
#Price: NoRidge>NridgHt>StoneBr>OldTown>Somerst=CollgCr>Veenker>Crawfor>Timber>Gilbert>Sawyer=Edwards=ClearCr>NWAmes>Mitchel>Blmngtn>BrkSide=SWISU>IDOTRR=Blueste=NPkVill>BrDale
Mapping_neightb = {'BrDale':1, 'MeadowV':2, 'NPkVill':2, 'Blueste':2, 'IDOTRR':2, 'Sawyer':2, 'SWISU':3, 'BrkSide':3, 'Blmngtn':4, 'Mitchel':5, 'NAmes':6, 'NWAmes':6, 'ClearCr':7, 'Edwards':8, 'SawyerW':8, 'Gilbert':9, 'Timber':10, 'Crawfor':11, 'Veenker':12, 'CollgCr':13, 'Somerst':13, 'OldTown':14, 'StoneBr':15, 'NridgHt':16, 'NoRidge':17}
test['Neighborhood'] = test['Neighborhood'].replace(Mapping_neightb)
#Price condition:RRAe<RRNe<Feedr<RRNn<PosA<PosN<RRAn<Artery<Norm
Mapping_condition1 = {'RRAe':1, 'RRNe':2, 'Feedr':3, 'RRNn':4, 'PosA':5, 'PosN':6, 'RRAn':7, 'Artery':8, 'Norm':9}
test['Condition1'] = test['Condition1'].replace(Mapping_condition1)
Mapping_condition2 = {'Artery':1, 'RRNn':2, 'RRAn':3, 'Feedr':4, 'RRAe':5, 'PosA':6, 'PosN':7, 'Norm':8}
test['Condition2'] = test['Condition2'].replace(Mapping_condition2)
#plt.bar(df['HouseStyle'], df['SalePrice'], width=0.1)
Mapping_Bldg = {'Duplex':1, '2fmCon':2, 'Twnhs':3, 'TwnhsE':4, '1Fam':5}
test['BldgType'] = test['BldgType'].replace(Mapping_Bldg)
Mapping_housestyle = {'1.5Unf':1, 'SFoyer':2, '2.5Unf':3, 'SLvl':4, '1.5Fin':5, '2.5Fin':6, '1Story':7, '2Story':8}
test['HouseStyle'] = test['HouseStyle'].replace(Mapping_housestyle)
#plt.bar(df['MasVnrType'], df['SalePrice'], width=0.1)
Mapping_RoofStyle = {'Gambrel':1, 'Mansard':1, 'Flat':1, 'Shed':1, 'Gable':2, 'Hip':2}
test['RoofStyle'] = test['RoofStyle'].replace(Mapping_RoofStyle)
Mapping_RoofMatl = {'Metal':1, 'Roll':1, 'ClyTile':1, 'WdShake':2, 'Membran':2, 'Tar&Grv':2, 'CompShg':3, 'WdShngl':3}
test['RoofMatl'] = test['RoofMatl'].replace(Mapping_RoofMatl)
Mapping_exterior1 = {'BrkComm':1, 'AsphShn':1, 'CBlock':1, 'AsbShng':1, 'Stone':2, 'ImStucc':2, 'Stucco':2, 'Plywood':2, 'WdShing':2, 'BrkFace':2, 'MetalSd':2, 'CemntBd':3, 'VinylSd':3, 'Wd Sdng':4, 'HdBoard':4}
test['Exterior1st'] = test['Exterior1st'].replace(Mapping_exterior1)
Mapping_exterior2 = {'Brk Cmn':1, 'AsphShn':1, 'AsbShng':1, 'Stone':1, 'CBlock':1, 'Other':2, 'Plywood':2,'Wd Shng':2 ,'Wd Sdng':3, 'BrkFace':2, 'Stucco':2, 'MetalSd':2, 'VinylSd':3, 'CmentBd':3, 'HdBoard':3, 'ImStucc':3}
test['Exterior2nd'] = test['Exterior2nd'].replace(Mapping_exterior2)
test['MasVnrType'].fillna(3, inplace=True)
Mapping_MasVnrType = {'BrkFace':3, 'Stone':2, 'BrkCmn':1}
test['MasVnrType'] = test['MasVnrType'].replace(Mapping_MasVnrType)
test['MasVnrArea'].fillna(0, inplace=True)
#plt.bar(df['Foundation'], df['SalePrice'], width=0.1)
Mapping_Foundation = {'Wood':1, 'Slab':1, 'Stone':1, 'BrkTil':2, 'CBlock':2, 'PConc':3}
test['Foundation'] = test['Foundation'].replace(Mapping_Foundation)
test['BsmtQual'].fillna(0, inplace=True)
test['BsmtCond'].fillna(0, inplace=True)
test['BsmtExposure'].fillna(0, inplace=True)
test['BsmtFinType1'].fillna(0, inplace=True)
test['BsmtFinType2'].fillna(0, inplace=True)
#plt.bar(df['Electrical'], df['SalePrice'], width=0.1)
mapping_heating = {'Grav':1, 'Wall':1, 'OthW':1, 'Floor':1, 'GasW':2, 'GasA':3}
test['Heating'] = test['Heating'].replace(mapping_heating)
test['CentralAir'] = test['CentralAir'].replace({'N':0, 'Y':1})
test['Electrical'].fillna('None', inplace=True)
#plt.bar(df['Functional'], df['SalePrice'], width=0.1)
test['Electrical'] = test['Electrical'].replace({'FuseF':1, 'FuseA':1, 'FuseP':1, 'Mix':1, 'None':1, 'SBrkr':2})
test['Functional'] = test['Functional'].replace({'Maj2':1, 'Sev':1, 'Min1':2, 'Maj1':2, 'Min2':2, 'Mod':3, 'Typ':4})
test['FireplaceQu'].fillna(0, inplace=True)
test['GarageType'].fillna('None', inplace=True)
test['GarageType'] = test['GarageType'].replace({'CarPort':1, 'None':1, '2Types':1, 'Basment':1, 'Detchd':2, 'BuiltIn':2, 'Attchd':3})
test['GarageFinish'].fillna('None', inplace=True)
#plt.bar(df['GarageFinish'], df['SalePrice'], width=0.1)
test['GarageFinish'] = test['GarageFinish'].replace({'None':0, 'RFn':1, 'Unf':1, 'Fin':1})
test['GarageYrBlt'].fillna(0, inplace=True)
test['GarageQual'].fillna(0, inplace=True)
test['GarageCond'].fillna(0, inplace=True)
#plt.bar(df['PavedDrive'], df['SalePrice'], width=0.1)
test['PavedDrive'] = test['PavedDrive'].replace({'Y':1, 'N':0, 'P':0})
test['PoolQC'].fillna(0, inplace=True)
test['Fence'].fillna('None', inplace=True)
#plt.bar(df['Fence'], df['SalePrice'], width=0.1)
test['Fence'] = test['Fence'].replace({'GdWo':1, 'GdPrv':1, 'MnWw':1, 'MnPrv':2, 'None':2})
test['MiscFeature'].fillna('None', inplace=True)
test['MiscFeature'] = test['MiscFeature'].replace({'None':1, 'Shed':0, 'Gar2':0, 'Othr':0, 'TenC':0})
#plt.bar(df['SaleCondition'], df['SalePrice'], width=0.1)
test['SaleType'] = test['SaleType'].replace({'COD':1, 'ConLD':1, 'CWD':1, 'ConLw':1, 'Con':1, 'Oth':1, 'ConLI':1, 'New':2, 'WD':2})
test['SaleCondition'] = test['SaleCondition'].replace({'AdjLand':1, 'Alloca':1, 'Family':1, 'Partial':2, 'Abnorml':2, 'Normal':2})
test['MSSubClass'] = test['MSSubClass'].replace({60:3, 20:3, 50:2, 75:2, 120:2, 30:1, 35:1, 40:1, 45:1, 70:1, 80:1, 85:1, 90:1, 160:1, 190:1, 180:1})
test['MSZoning'].fillna(test['MSZoning'].mean(), inplace=True)
test['Utilities'].fillna(test['Utilities'].mean(), inplace=True)
test['Exterior1st'].fillna(test['Exterior1st'].mean(), inplace=True)
test['Exterior2nd'].fillna(test['Exterior2nd'].mean(), inplace=True)
test['BsmtFinSF1'].fillna(test['BsmtFinSF1'].mean(), inplace=True)
test['BsmtFinSF2'].fillna(test['BsmtFinSF2'].mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

In [13]:
#Submission
submission = test[['Id']]
test.drop('Id', axis=1, inplace=True)
test_scaled = robust_scaler.fit_transform(test)
y_submission = model_lasso.predict(test_scaled)
submission['SalePrice'] = np.expm1(y_submission)
submission.to_csv("LassoReg_Encoding.csv", index=False)

/tmp/ipykernel_20/402812023.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['SalePrice'] = np.expm1(y_submission)
